In [9]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from flowprintOptimal.sekigo.core.flowConfig import FlowConfig
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from flowprintOptimal.sekigo.flowUtils.commons import loadFlows, saveFlows
from flowprintOptimal.sekigo.flowUtils.sampler import FixedLengthSampler
from flowprintOptimal.sekigo.core.flowRepresentation import FlowRepresentation
from typing import List
from flowprintOptimal.sekigo.flowUtils.flowDatasets import ActivityDataset
from torch.utils.data import DataLoader

In [2]:
class Agent(nn.Module):
    def __init__(self,lstm_input_size,lstm_hidden_size,num_classes,lr = .001):
        super().__init__()
        self.lstm = nn.LSTM(input_size= lstm_input_size,hidden_size= lstm_hidden_size,batch_first= True)
        self.linear = nn.Linear(in_features= lstm_hidden_size,out_features = num_classes)
        self.softmax = nn.Softmax(dim= -1)

        self.optimizer = torch.optim.Adam(params= self.parameters(), lr= lr)
        # TODO look at the projection size
    
    def forward(self,X):
        """
        X is the timeseries input of shape 
        (BS,Seq len, lstm_input_size)

        The output is of shape (BS,seq len,num_classes)
        """
        lstm_out, _ = self.lstm(X)
        return self.softmax(self.linear(lstm_out))
    

    @staticmethod
    def selectMaxAction(probs):
        """
        probs is of shape (BS,num_actions)
        """
        return torch.argmax(input= probs, dim= -1).to("cpu").numpy()

    @staticmethod
    def sampleAction(probs):
        """
        probs is of shape (BS,num_actions)
        """

        return torch.multinomial(probs,1)
        


In [3]:
class TSEnvironment:
    def __init__(self,flow_config : FlowConfig,max_prediction_length_in_seconds,min_prediction_length_in_seconds):
        self.flow_config = flow_config
        # length 10 secs grain = .1
        self.max_prediction_length = int(max_prediction_length_in_seconds/flow_config.grain)
        self.min_prediction_length = int(min_prediction_length_in_seconds/flow_config.grain)

        self.max_reward = 1
        self.max_penalty = -1

    def rewardFunction(self,predicted,target,prediction_length,num_classes):
        """
        This is a vannila reward, but I have to take care of out od distrubation reward as well.
        It returns reward and is ended flag
        """
        if predicted == num_classes -1:
            # this is a signal to wait
            return 0,False
        else:
            if predicted == target:
                return self.max_reward*(1 - (prediction_length - self.min_prediction_length)/(self.max_prediction_length - self.min_prediction_length)),True
            else:
                return self.max_penalty,True
    
    @staticmethod
    def getFutureReturns(rewards,gamma = .99):
        returns = [0]*len(rewards)
        returns[-1] = rewards[-1]
        for i in range(len(rewards)-2,-1,-1):
            returns[i] = returns[i+1]*gamma + rewards[i]
        
        return returns

    def generateEpisodesAndTrain(self,time_series : np.ndarray,targets : np.ndarray,agent : Agent):
        """
        Here time_series is a single sample of shape
        (BS,seq len, features)
        target is of shape 
        (BS,1)
        """

        def processSingleEpisode(probs : torch.Tensor,target):
            """
            in this case probs is (seq len,num_classes)
            As soon as we encounter a prediction we give the reward and terminate

            target of shape (1)
            """
            num_classes = probs.shape[1]
            sampled_actions = Agent.sampleAction(probs= probs.clone().detach())  # sampled actions of shape (seq len, 1)
            rewards = []
            action_probs = []
            for i in range(self.min_prediction_length,self.max_prediction_length + 1):
                action = sampled_actions[i]
                reward, is_ended = self.rewardFunction(predicted= action, target= target, num_classes= num_classes, prediction_length= i)

                rewards.append(reward)
                action_probs.append(probs[i,sampled_actions[i]])

                if is_ended == True:
                    break
            
            future_returns = TSEnvironment.getFutureReturns(rewards= rewards)
            return action_probs,future_returns
            


        assert time_series.shape[1] >= self.min_prediction_length

        series = torch.tensor(time_series).to(device)


        probs = agent(series) # probs (BS,seq len, num_classes)

        action_probs,future_returns = [],[]

        for i in range(len(probs)):
            action_probs_,future_returns_ = processSingleEpisode(probs= probs[i],target= targets[i])
            action_probs.extend(action_probs_)
            future_returns.extend(future_returns_)


        action_probs = torch.cat(action_probs)
        future_returns = torch.tensor(future_returns).to(device)

        loss = -(torch.log(action_probs)*future_returns).mean()
        
        loss.backward()
        agent.optimizer.step()
        agent.optimizer.zero_grad()
        

        

        




    



In [4]:
flow_config = FlowConfig(grain= 1,band_thresholds= [1250])

In [5]:
ts_environ = TSEnvironment(flow_config= flow_config,max_prediction_length_in_seconds= 40, min_prediction_length_in_seconds= 5)
agent = Agent(lstm_hidden_size= 10,lstm_input_size= 4,num_classes= 3)

In [6]:
ts_environ.generateEpisodes(time_series= np.random.rand(64,20,4).astype(np.float32),targets= torch.zeros(64,1),agent= agent)

In [4]:
flows = loadFlows(path= "/Users/rushi/Desktop/UNSW/data/VNAT/flowStore/vnatflows1second.json")

In [5]:
corrected_flows = FixedLengthSampler.sampleAndCutToLength(data= flows,flow_config= flows[0].flow_config,required_length_in_seconds= 40,max_cuts= 10, min_activity_for_start_point= 20)

None


In [6]:
activity_dataset = ActivityDataset(flows= corrected_flows)

In [7]:
activity_dataset[0]

{'flow': array([[0.        , 0.25215995, 0.        , 0.11519994, 0.0959999 ,
         0.        , 0.25386658, 0.        , 0.21631996, 0.15999998,
         0.1378909 , 0.        , 0.        , 0.        , 0.1045333 ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.20036922, 0.13530872, 0.11825321, 0.32106664,
         0.10879996, 0.23466665, 0.09599997, 0.09599998, 0.        ,
         0.25874284, 0.10879996, 0.21270587, 0.09599995, 0.22399998,
         0.        , 0.20479995, 0.1221703 , 0.11471304, 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.9594073 , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.90725916, 0.89066637, 0.96533237,
         0.        , 0.96533237, 0.        , 0.        , 0.        ,
         0.96533285, 0.  

In [10]:
loader = DataLoader(dataset= activity_dataset, batch_size= 64)

In [11]:
for i in loader:
    break

torch.Size([64, 4, 40])